# Dwell Time Sequential Control Design - Mode 2

In [1]:
# Import packages
import numpy as np
import scipy as sp
import cvxpy as cp
import numpy.linalg as npla
import scipy.linalg as spla
import matplotlib.pyplot as plt
import IPython.display as ipdi
import lib as lb

In [2]:
# System state-space definition
A_list = [np.array([[+1, +1, -1], [0.0, +1, +1], [0.0, 0.0, +1]])]
A_list += [np.array([[+1, +1, 0.0], [0.0, +1, -1], [0.0, 0.0, +1]])]
A_list += [np.array([[+1, -1, 0.0], [0.0, +1, 0.0], [0.0, 0.0, +1]])]
B_list = [np.array([[+1, 0.0], [+1, +1], [0.0, +1]])]
B_list += [np.array([[0.0, 0.0], [+1, 0.0], [0.0, +1]])]
B_list += [np.array([[0.0, +1], [0.0, 0.0], [0.0, +1]])]

# Dwell time lower bounds for subsystems
D1 = 1
D2 = 12
D3 = 5

# Weighting matrices
Q = np.identity(3)
R = np.identity(2)

# System dimensions
nxn = np.shape(A_list[0])
nxm = np.shape(B_list[0])
mxn = nxm[::-1]

# Switching sequence for simulation with known sequence
#sw = [np.random.randint(3) for i in range(50)]
#sw = [np.random.choice(np.arange(3), p=[0.8, 0.1, 0.1]) for i in range(50)]
sw = np.tile([0, 1, 2, 2, 0], 10)

# Set of possible switching sequences
Is = [(0, 1), (1, 2), (2, 0)]

# Initial state
x_0 = np.array([[25], [-50], [100]])

# Matplotlib settings
ipdi.set_matplotlib_formats('svg')

## Design control mode 2 - $\Delta_{2}=10$

In [3]:
# Optimization variables
X2 = cp.Variable(shape=nxn, PSD=True)
K2 = cp.Variable(shape=mxn, PSD=False)

In [4]:
# Constraints
A2 = A_list[1]
B2 = B_list[1]
Constraints = [cp.bmat([[X2, (A2*X2 + B2*K2).T, (spla.sqrtm(Q)*X2).T, (spla.sqrtm(R)*K2).T],
                        [A2*X2 + B2*K2, X2, np.zeros(nxn), np.zeros(nxm)],
                        [spla.sqrtm(Q)*X2, np.zeros(nxn), np.identity(nxn[0]), np.zeros(nxm)],
                        [spla.sqrtm(R)*K2, np.zeros(mxn), np.zeros(mxn), np.identity(nxm[1])]]) >> 0]

In [5]:
# Objective function
Objective = cp.Minimize(-cp.trace(X2))

In [6]:
# Problem formulation
Problem = cp.Problem(Objective, Constraints)
Problem.solve(solver=cp.CVXOPT, eps=1e-12, verbose=True)

     pcost       dcost       gap    pres   dres   k/t
 0:  0.0000e+00 -6.6395e+00  2e+01  2e+00  4e+00  1e+00
 1: -1.6595e+00 -2.7233e+00  4e+00  3e-01  8e-01  4e-01
 2: -1.5386e+00 -1.6980e+00  5e-01  4e-02  1e-01  3e-02
 3: -1.4629e+00 -1.4722e+00  3e-02  2e-03  6e-03  1e-03
 4: -1.4618e+00 -1.4626e+00  2e-03  2e-04  5e-04  1e-04
 5: -1.4614e+00 -1.4615e+00  3e-04  3e-05  6e-05  2e-05
 6: -1.4614e+00 -1.4614e+00  6e-06  5e-07  1e-06  3e-07
 7: -1.4614e+00 -1.4614e+00  3e-07  2e-08  6e-08  2e-08
Optimal solution found.


-1.4614000195857753

In [7]:
# Compute Lyapunov function mode 2
P2 = spla.inv(X2.value)
print("The Lyapunov P2 is equal to: \n {}".format(P2))

The Lyapunov P2 is equal to: 
 [[ 2.92714145  2.35822028 -0.3883925 ]
 [ 2.35822028  4.5954421  -1.26168039]
 [-0.3883925  -1.26168039  2.64265093]]


In [8]:
# Compute the eigenvalues of P2
(eig_2, vec_2) = spla.eig(P2)
print("The eigenvalues of P2 are equal to: \n {}".format(eig_2))

The eigenvalues of P2 are equal to: 
 [6.65769767+0.j 1.12080644+0.j 2.38673037+0.j]


In [9]:
# Compute state feedback gain mode 2
F2 = K2.value @ P2
print("The feedback gain F2 is equal to: \n {}".format(F2))

The feedback gain F2 is equal to: 
 [[-0.43110884 -1.23724189  0.87328944]
 [-0.04273072  0.02446158 -0.76930923]]


In [10]:
# Closed-loop system dynamics in mode 2
Acl2 = A2 + B2@F2
print("The closed-loop dynamics in mode 2 is: \n {}".format(Acl2))

The closed-loop dynamics in mode 2 is: 
 [[ 1.          1.          0.        ]
 [-0.43110884 -0.23724189 -0.12671056]
 [-0.04273072  0.02446158  0.23069077]]


In [11]:
# Compute the eigenvalues of closed-loop subsystem 2
(eig_cl2, vec_cl2) = spla.eig(Acl2)
print("The eigenvalues of Acl2 are equal to: \n {}".format(eig_cl2))

The eigenvalues of Acl2 are equal to: 
 [0.30568617+0.21422577j 0.30568617-0.21422577j 0.38207653+0.j        ]


## Design control mode 3 - $\Delta_{3}=2$

In [12]:
# Optimization variables
X3 = cp.Variable(shape=nxn, PSD=True)
K3 = cp.Variable(shape=mxn, PSD=False)

In [13]:
# Constraints
A3 = A_list[2]
B3 = B_list[2]
Constraints = [cp.bmat([[X3, (A3*X3 + B3*K3).T, (spla.sqrtm(Q)*X3).T, (spla.sqrtm(R)*K3).T],
                        [A3*X3 + B3*K3, X3, np.zeros(nxn), np.zeros(nxm)],
                        [spla.sqrtm(Q)*X3, np.zeros(nxn), np.identity(nxn[0]), np.zeros(nxm)],
                        [spla.sqrtm(R)*K3, np.zeros(mxn), np.zeros(mxn), np.identity(nxm[1])]]) >> 0]
Constraints += [cp.bmat([[P2, npla.matrix_power(Acl2, D2).T],
                         [npla.matrix_power(Acl2, D2), X3]]) >> 0]

In [14]:
# Objective function
Objective = cp.Minimize(-cp.trace(X3))

In [15]:
# Problem formulation
Problem = cp.Problem(Objective, Constraints)
Problem.solve(solver=cp.SCS, eps=7.5e-5, verbose=True)

----------------------------------------------------------------------------
	SCS v2.1.1 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 57
eps = 7.50e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 10, rho_x = 1.00e-03
Variables n = 12, constraints m = 93
Cones:	sd vars: 93, sd blks: 3
Setup time: 3.20e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 3.99e+19  5.86e+17  9.97e-01 -1.81e+19 -2.37e+16  6.55e+20  2.57e-03 
   100| 2.73e-03  1.16e-02  2.16e-02 -9.52e-01 -8.91e-01  1.40e-15  1.27e-02 
   200| 1.03e-03  3.65e-03  5.66e-03 -8.24e-01 -8.39e-01  4.11e-16  3.74e-02 
   300| 1.28e-03  4.35e-03  5.44e-05 -8.25e-

-0.7791264313754219

In [16]:
# Compute Lyapunov function mode 3
P3 = spla.inv(X3.value)
print("The Lyapunov P3 is equal to: \n {}".format(P3))

The Lyapunov P3 is equal to: 
 [[ 3.84380943e+01 -1.05115336e+03 -3.70716781e+01]
 [-1.05115336e+03  4.07325210e+04  1.05068704e+03]
 [-3.70716781e+01  1.05068704e+03  3.84373398e+01]]


In [17]:
# Compute the eigenvalues of P3
(eig_3, vec_3) = spla.eig(P3)
print("The eigenvalues of P3 are equal to: \n {}".format(eig_3))

The eigenvalues of P3 are equal to: 
 [4.07867779e+04+0.j 1.36602940e+00+0.j 2.12525190e+01+0.j]


In [18]:
# Compute state feedback gain mode 3
F3 = K3.value @ P3
print("The feedback gain F2 is equal to: \n {}".format(F3))

The feedback gain F2 is equal to: 
 [[-0.03981736  1.48848784  0.03973852]
 [-0.39853455  1.73005077 -0.3335413 ]]


In [19]:
# Closed-loop system dynamics in mode 3
Acl3 = A3 + B3@F3
print("The closed-loop dynamics in mode 3 is: \n {}".format(Acl3))

The closed-loop dynamics in mode 3 is: 
 [[ 0.60146545  0.73005077 -0.3335413 ]
 [ 0.          1.          0.        ]
 [-0.39853455  1.73005077  0.6664587 ]]


In [20]:
# Compute the eigenvalues of closed-loop subsystem 3
(eig_cl3, vec_cl3) = spla.eig(Acl3)
print("The eigenvalues of Acl3 are equal to: \n {}".format(eig_cl3))

The eigenvalues of Acl3 are equal to: 
 [0.26792416+0.j 1.        +0.j 1.        +0.j]


## Design control mode 1 - $\Delta_{1}=5$

In [21]:
# Optimization variables
X1 = cp.Variable(shape=nxn, PSD=True)
K1 = cp.Variable(shape=mxn, PSD=False)

In [22]:
# Constraints
A1 = A_list[0]
B1 = B_list[0]
Constraints = [cp.bmat([[X1, (A1*X1 + B1*K1).T, (spla.sqrtm(Q)*X1).T, (spla.sqrtm(R)*K1).T],
                        [A1*X1 + B1*K1, X1, np.zeros(nxn), np.zeros(nxm)],
                        [spla.sqrtm(Q)*X1, np.zeros(nxn), np.identity(nxn[0]), np.zeros(nxm)],
                        [spla.sqrtm(R)*K1, np.zeros(mxn), np.zeros(mxn), np.identity(nxm[1])]]) >> 0]
Constraints += [cp.bmat([[P2, npla.matrix_power(Acl2, D2).T],
                         [npla.matrix_power(Acl2, D2), X1]]) >> 0]
Constraints += [cp.bmat([[P3, npla.matrix_power(Acl3, D3).T],
                         [npla.matrix_power(Acl3, D3), X1]]) >> 0]

In [23]:
# Objective function
Objective = cp.Minimize(-cp.trace(X1))

In [24]:
# Problem formulation
Problem = cp.Problem(Objective, Constraints)
Problem.solve(solver=cp.CVXOPT, eps=1e-12, verbose=True)

     pcost       dcost       gap    pres   dres   k/t
 0:  0.0000e+00 -5.0899e+04  5e+04  1e-04  6e+00  1e+00
 1: -5.0206e+00 -4.9159e+04  5e+04  1e-04  6e+00  4e+00
 2: -1.5527e+00 -4.8545e+04  5e+04  1e-04  6e+00  1e+01
 3: -4.7383e+00 -4.7851e+04  5e+04  1e-04  6e+00  3e+01
 4: -3.4520e+00 -3.8653e+04  4e+04  1e-04  5e+00  9e+01
 5: -1.5692e+00 -7.6778e+02  8e+02  2e-06  9e-02  2e+00
 6: -1.5161e+00 -2.8928e+01  3e+01  7e-08  3e-03  6e-02
 7: -1.5534e+00 -4.6733e+00  3e+00  8e-09  4e-04  8e-03
 8: -1.6632e+00 -4.0346e+00  2e+00  6e-09  3e-04  2e-02
 9: -1.7045e+00 -2.5001e+00  8e-01  2e-09  1e-04  8e-03
10: -1.7256e+00 -1.9859e+00  3e-01  7e-10  3e-05  4e-03
11: -1.7345e+00 -1.7729e+00  4e-02  1e-10  5e-06  1e-03
12: -1.7384e+00 -1.7442e+00  6e-03  2e-11  7e-07  2e-04
13: -1.7387e+00 -1.7404e+00  2e-03  5e-12  2e-07  7e-05
14: -1.7388e+00 -1.7392e+00  4e-04  1e-12  5e-08  2e-05
15: -1.7388e+00 -1.7390e+00  1e-04  3e-13  2e-08  5e-06
16: -1.7389e+00 -1.7389e+00  3e-05  8e-14  4e-09  

-1.7388537890085198

In [25]:
# Compute Lyapunov function mode 1
P1 = spla.inv(X1.value)
print("The Lyapunov P1 is equal to: \n {}".format(P1))

The Lyapunov P1 is equal to: 
 [[ 1.70863815  0.42957555 -0.88682483]
 [ 0.42957555  1.80719839 -0.39168736]
 [-0.88682483 -0.39168736  2.85249333]]


In [26]:
# Compute the eigenvalues of P1
(eig_1, vec_1) = spla.eig(P1)
print("The eigenvalues of P1 are equal to: \n {}".format(eig_1))

The eigenvalues of P1 are equal to: 
 [3.51433779+0.j 1.15519843+0.j 1.69879365+0.j]


In [27]:
# Compute state feedback gain mode 1
F1 = K1.value @ P1
print("The feedback gain F1 is equal to: \n {}".format(F1))

The feedback gain F1 is equal to: 
 [[-0.42957275 -0.80719767  0.39168416]
 [ 0.17818387 -0.03788955 -0.96566465]]


In [28]:
# Closed-loop system dynamics in mode 1
Acl1 = A1 + B1@F1
print("The closed-loop dynamics in mode 1 is: \n {}".format(Acl1))

The closed-loop dynamics in mode 1 is: 
 [[ 0.57042725  0.19280233 -0.60831584]
 [-0.25138888  0.15491278  0.42601951]
 [ 0.17818387 -0.03788955  0.03433535]]


In [29]:
# Compute the eigenvalues of closed-loop subsystem 1
(eig_cl1, vec_cl1) = spla.eig(Acl1)
print("The eigenvalues of Acl1 are equal to: \n {}".format(eig_cl1))

The eigenvalues of Acl1 are equal to: 
 [0.25576906+0.30647009j 0.25576906-0.30647009j 0.24813725+0.j        ]


## Dwell time switch computation

In [30]:
# Switch from subsystem 1 to subsystem 2
flag = True
D12 = 0
while flag:
    D12 += 1
    if np.all(npla.eigvals(P1 - npla.matrix_power(Acl1, D12).T@P2@npla.matrix_power(Acl1, D12)) > 0):
        flag = False
print("The minimum dwell time from mode 1 to 2 is: {}".format(D12))

The minimum dwell time from mode 1 to 2 is: 1


In [31]:
# Switch from subsystem 2 to subsystem 1
flag = True
D21 = 0
while flag:
    D21 += 1
    if np.all(npla.eigvals(P2 - npla.matrix_power(Acl2, D21).T@P1@npla.matrix_power(Acl2, D21)) > 0):
        flag = False
print("The minimum dwell time from mode 2 to 1 is: {}".format(D21))

The minimum dwell time from mode 2 to 1 is: 1


In [32]:
# Switch from subsystem 2 to subsystem 3
flag = True
D23 = 0
while flag:
    D23 += 1
    if np.all(npla.eigvals(P2 - npla.matrix_power(Acl2, D23).T@P3@npla.matrix_power(Acl2, D23)) > 0):
        flag = False
print("The minimum dwell time from mode 2 to 3 is: {}".format(D23))

The minimum dwell time from mode 2 to 3 is: 7


In [33]:
# Switch from subsystem 3 to subsystem 1
flag = True
D31 = 0
while flag:
    D31 += 1
    if np.all(npla.eigvals(P3 - npla.matrix_power(Acl3, D31).T@P1@npla.matrix_power(Acl3, D31)) > 0):
        flag = False
print("The minimum dwell time from mode 3 to 1 is: {}".format(D31))

The minimum dwell time from mode 3 to 1 is: 1


## Simulation with switching system dynamics in $\mathcal{I}_{s}$

In [34]:
inv_a = spla.lapack.dpotri(4*np.identity(3))

In [35]:
inv_a

(array([[ 0.0625,  0.    , -0.    ],
        [ 0.    ,  0.0625, -0.    ],
        [ 0.    ,  0.    ,  0.0625]]), 0)

In [36]:
spla.inv(X1.value)

array([[ 1.70863815,  0.42957555, -0.88682483],
       [ 0.42957555,  1.80719839, -0.39168736],
       [-0.88682483, -0.39168736,  2.85249333]])

In [37]:
inv_a[0]*4*np.identity(3)

array([[ 0.25,  0.  , -0.  ],
       [ 0.  ,  0.25, -0.  ],
       [ 0.  ,  0.  ,  0.25]])

In [38]:
cp.installed_solvers()

['CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'OSQP', 'SCS']

In [39]:
print("F1 is equal to: \n {} \n".format(F1))
print("F2 is equal to: \n {} \n".format(F2))
print("F3 is equal to: \n {} \n".format(F3))

F1 is equal to: 
 [[-0.42957275 -0.80719767  0.39168416]
 [ 0.17818387 -0.03788955 -0.96566465]] 

F2 is equal to: 
 [[-0.43110884 -1.23724189  0.87328944]
 [-0.04273072  0.02446158 -0.76930923]] 

F3 is equal to: 
 [[-0.03981736  1.48848784  0.03973852]
 [-0.39853455  1.73005077 -0.3335413 ]] 



In [40]:
print(P1)

[[ 1.70863815  0.42957555 -0.88682483]
 [ 0.42957555  1.80719839 -0.39168736]
 [-0.88682483 -0.39168736  2.85249333]]


In [41]:
print(P2)

[[ 2.92714145  2.35822028 -0.3883925 ]
 [ 2.35822028  4.5954421  -1.26168039]
 [-0.3883925  -1.26168039  2.64265093]]


In [42]:
print(P3)

[[ 3.84380943e+01 -1.05115336e+03 -3.70716781e+01]
 [-1.05115336e+03  4.07325210e+04  1.05068704e+03]
 [-3.70716781e+01  1.05068704e+03  3.84373398e+01]]
